<a href="https://colab.research.google.com/github/jafr0822/MNA_ProyectoIntegrador-Equipo7/blob/main/Avance3_7_Equipo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto Integrador


## Tecnológico de Monterrey
### Maestría en Inteligencia Artificial Aplicada (MNA)
#### Avance 3
#### Equipo 7


* Jorge Arturo Federico Rivera – A01250724
* Marco Antonio Vázquez Morales – A01793704
* Alejandro Jesús Vázquez Navarro - A01793146

### Baseline

Proyecto:

*Modelo clasificador de multimorbilidad maternal y predictor de desenlaces perinatales a partir de datos clínicos metabólicos, genéticos y nutricionales de mujeres mexicanas*

19 de mayo de 2024

# 1. Algoritmo

El problema escogido será tratad como aprendizaje supervisado. Nos enfocaremos en la predicción de una variable dependiente "EscalaRiesgo" que contendrá 3 valores:
- A. Para alto riesgo (la presencia de 3 o 4 factores multimorbilidad)
- B. Para medio riesgo (presencia de 1 o 2 factores multimorbilidad)
- C. Para riesgo nulo. (sin la presencia de factores de multimorbilidad)

En esta entrega utilizaremos un algoritmo de random forest por las siguientes causas:
- **Precisión**. Los árboles de bosque aleatorios proporcionan buenos resultados cuando tenemos datos con muchas características.
- **Manejo del sobreajuste.** Mediante el promedio de los resultados de los árboles, los bosques aleatorios reducen el riesgo de sobreajuste en comparación con un único árbol de decisión.
- **Heterogeneidad.** Funcionan muy buen con datos tanto numéricos como categóricos
- Son mucho menos sensibles a datos ruidosos.
- **Generalización.** Dan muy buenos resultados en térrminos de generalización para nuevas observaciones
